Bokeh offers many controls for interactive plots. In addition, Bokeh allows for further customization with CustomJS callbacks, which allow users to write unique behaviors in JavaScript for specialized cases that can’t be solved with the existing Bokeh widgets. 

Selection tools emit events that can drive useful callbacks. Below, a callback for SelectionGeometry uses the BoxSelectTool geometry (accessed via the geometry field of the cb_data callback object), in order to update a Rect glyph.

In [1]:
from bokeh.events import SelectionGeometry
from bokeh.models import ColumnDataSource, CustomJS, Rect
from bokeh.plotting import figure, output_notebook, show

# Enable the Bokeh output in the Jupyter notebook
output_notebook()

source = ColumnDataSource(data=dict(x=[], y=[], width=[], height=[]))

callback = CustomJS(args=dict(source=source), code="""
    const geometry = cb_obj['geometry']
    const data = source.data

    // calculate Rect attributes
    const width = geometry['x1'] - geometry['x0']
    const height = geometry['y1'] - geometry['y0']
    const x = geometry['x0'] + width/2
    const y = geometry['y0'] + height/2

    // update data source with new Rect attributes
    data['x'].push(x)
    data['y'].push(y)
    data['width'].push(width);
    data['height'].push(height)

    // emit update of data source
    source.change.emit()
""")

p = figure(width=400, height=400, tools="box_select",
           title="Select Below", x_range=(0, 1), y_range=(0, 1))

rect = Rect(x='x', y='y', width='width', height='height',
            fill_alpha=0.3, fill_color='#009933')

p.add_glyph(source, rect, selection_glyph=rect, nonselection_glyph=rect)

p.js_on_event(SelectionGeometry, callback)

show(p)

Loading BokehJS ...